# CORD Search Engine

## CORD Library

In [5]:
%load_ext autoreload
%autoreload 2
from cord.cord19 import *
from cord.core import parallel, ifnone
pd.options.display.max_colwidth = 200

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dwight\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Reading the Research Papers

## From Paths

In [9]:
research_papers = ResearchPapers.from_data_dir()
#research_papers = ResearchPapers.from_pickle()

Loading metadata from data\CORD-19-research-challenge
Cleaning metadata
Fixing dates that are a list e.g. "['2020-02-05', '2020-02']"
Fixing dates with the seasons e.g. "2014 Autumn"
Fixing dates like "2016 Nov 9 Jan-Feb"
Fixing dates like "2012 Jan-Mar"
Converting dates like "2020 Apr 13"
Converting Dates like "2020 Apr"
Converting dates like "2020"
Converting Dates like "2020-01-21"
Indexing research papers
Finished Indexing in 85.0 seconds


In [7]:
research_papers.save()

Saving to data\ResearchPapers.pickle


## From Pickle

In [8]:
%time research_papers = ResearchPapers.from_pickle()

Wall time: 2.69 s


In [ ]:
research_papers.metadata.doi.isnull()

## Search Bar

In [10]:
research_papers.searchbar('vaccine transmission')

interactive(children=(Text(value='vaccine transmission', description='SearchTerms'), Output()), _dom_classes=(…

## Search

In [4]:
research_papers.search('treatment sars-cov-2', 
                       start_date='2020-01-30', 
                       end_date='2020-02-15',
                       covid_related=True)

In [13]:
research_papers.search('remdesivir')

In [36]:
research_papers.searchbar()

interactive(children=(Text(value='cruise ship', description='SearchTerms'), Output()), _dom_classes=('widget-i…